In [2]:
import pandas as pd
import mysql.connector

In [4]:
# Cargar CSV´s

df_balance = pd.read_csv("../data/processed/balance/balance-electrico-limpio.csv")

# df_demanda_evolucion = pd.read_csv("data/processed/demanda/demanda_evolucion_2019a2023_limpio.csv")
# df_demanda_ire_general = pd.read_csv("data/processed/demanda/demanda_ire_general_2019a2023_limpio.csv")
# df_demanda_ire_servicios = pd.read_csv("data/processed/demanda/demanda_ire_servicios_2019a2023_limpio.csv")

# df_estructura_generacion = pd.read_csv("../data/processed/generacion/estructura_generacion_limpio.csv")
# df_evolucion_renovable_no_renovable = pd.read_csv("../data/processed/generacion/evolucion_renovable_norenovable_limpio.csv")

# df_fronteras = pd.read_csv("../data/processed/intercambio/fronteras_limpio.csv")
# df_enlace_baleares = pd.read_csv("../data/processed/intercambio/enlace_baleares_limpio.csv")

In [5]:
df_balance.head()

,fecha,valor,tipo,energia,region
0,2019-01-01,53.722120,Hidráulica,Renovable,peninsular
1,2019-01-02,61.294768,Hidráulica,Renovable,peninsular
2,2019-01-03,82.981951,Hidráulica,Renovable,peninsular
3,2019-01-04,94.301940,Hidráulica,Renovable,peninsular
4,2019-01-05,58.822692,Hidráulica,Renovable,peninsular


In [ ]:
# df_estructura_generacion.head()


,value,percentage,datetime,pais,año
0,-3181.766,0.5,2019-01-01,andorra-frontera,2019
1,-3959.886,0.5,2019-01-02,andorra-frontera,2019
2,-4407.091,0.5,2019-01-03,andorra-frontera,2019
3,-4373.999,0.5,2019-01-04,andorra-frontera,2019
4,-3870.764,0.5,2019-01-05,andorra-frontera,2019


In [ ]:
# df_evolucion_renovable_no_renovable.head()


,datetime,value,percentage,indicator,region,type,year
0,2019-01-01T00:00:00.000+01:00,169351.7695,0.308096,Renovable,peninsular,Renovable,2019
1,2019-01-02T00:00:00.000+01:00,294297.6145,0.414504,Renovable,peninsular,Renovable,2019
2,2019-01-03T00:00:00.000+01:00,229057.3825,0.320167,Renovable,peninsular,Renovable,2019
3,2019-01-04T00:00:00.000+01:00,210376.4825,0.279131,Renovable,peninsular,Renovable,2019
4,2019-01-05T00:00:00.000+01:00,214172.6470,0.315429,Renovable,peninsular,Renovable,2019


In [ ]:
# df_fronteras.head()

,value,percentage,datetime,pais,año
0,-208.239,0.003465,2019-01-01,francia-frontera,2019
1,-2325.553,0.036857,2019-01-02,francia-frontera,2019
2,-11123.719,0.243767,2019-01-03,francia-frontera,2019
3,-17758.093,0.397978,2019-01-04,francia-frontera,2019
4,-4267.317,0.102386,2019-01-05,francia-frontera,2019


In [ ]:
# df_enlace_baleares.head()

,value,percentage,datetime,pais,año
0,-3181.766,0.5,2019-01-01,andorra-frontera,2019
1,-3959.886,0.5,2019-01-02,andorra-frontera,2019
2,-4407.091,0.5,2019-01-03,andorra-frontera,2019
3,-4373.999,0.5,2019-01-04,andorra-frontera,2019
4,-3870.764,0.5,2019-01-05,andorra-frontera,2019


In [13]:
# Conexión MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password='rootroot',
    database="redes_electricas"
)

cursor = conn.cursor()



In [11]:
nuevo_orden = ['fecha', 'tipo', 'energia', 'region', 'valor']
df_balance = df_balance[nuevo_orden]

df_balance.head()

,fecha,tipo,energia,region,valor
0,2019-01-01,Hidráulica,Renovable,peninsular,53.722120
1,2019-01-02,Hidráulica,Renovable,peninsular,61.294768
2,2019-01-03,Hidráulica,Renovable,peninsular,82.981951
3,2019-01-04,Hidráulica,Renovable,peninsular,94.301940
4,2019-01-05,Hidráulica,Renovable,peninsular,58.822692


In [ ]:
# Subir datos balance
for _, row in df_balance.iterrows():
    cursor.execute("""
    INSERT INTO balance (fecha, tipo, energia, region, valor)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        tipo=VALUES(tipo),
        energia=VALUES(energia),
        valor=VALUES(valor)
""", tuple(row))

conn.commit()
cursor.close()
conn.close()

In [ ]:
# Subir datos demanda


In [ ]:
# Subir datos generacion


In [ ]:
# Subir datos intercambio
